In [1]:
class Node:
    def __init__(self, data, depth=0, lchild=None, rchild=None):
        self.data = data
        self.depth = depth
        self.lchild = lchild
        self.rchild = rchild


class KdTree:
    def __init__(self):
        self.KdTree = None
        self.n = 0
        self.nearest = None

    def create(self, dataSet, depth=0):
        if len(dataSet) > 0:
            m, n = np.shape(dataSet)
            self.n = n - 1
            axis = depth % self.n
            mid = int(m / 2)
            dataSetcopy = sorted(dataSet, key=lambda x: x[axis])
            node = Node(dataSetcopy[mid], depth)
            if depth == 0:
                self.KdTree = node
            node.lchild = self.create(dataSetcopy[:mid], depth+1)
            node.rchild = self.create(dataSetcopy[mid+1:], depth+1)
            return node
        return None

    def preOrder(self, node):
        if node is not None:
            print(node.depth, node.data)
            self.preOrder(node.lchild)
            self.preOrder(node.rchild)

    def search(self, x, count=1):
        nearest = []
        for i in range(count):
            nearest.append([-1, None])
        self.nearest = np.array(nearest)

        def recurve(node):
            if node is not None:
                axis = node.depth % self.n
                daxis = x[axis] - node.data[axis]
                if daxis < 0:
                    recurve(node.lchild)
                else:
                    recurve(node.rchild)

                dist = sqrt(sum((p1 - p2) ** 2 for p1, p2 in zip(x, node.data)))
                for i, d in enumerate(self.nearest):
                    if d[0] < 0 or dist < d[0]:
                        self.nearest = np.insert(self.nearest, i, [dist, node], axis=0)
                        self.nearest = self.nearest[:-1]
                        break

                n = list(self.nearest[:, 0]).count(-1)
                if self.nearest[-n-1, 0] > abs(daxis):
                    if daxis < 0:
                        recurve(node.rchild)
                    else:
                        recurve(node.lchild)

        recurve(self.KdTree)

        knn = self.nearest[:, 1]
        belong = []
        for i in knn:
            belong.append(i.data[-1])
        b = max(set(belong), key=belong.count)

        return self.nearest, b

In [2]:
import numpy as np
from math import sqrt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mpl_toolkits.axisartist.axislines as axislines
import matplotlib as mpl
from sklearn import preprocessing
min_max_scaler=preprocessing.MinMaxScaler()
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
kf=KFold(n_splits=10)


def DDF(df):
    data2=np.array(df.drop('label',axis=1))
    dmax=[]
    for m in range(0,len(data2)-1):
        for n in range(m+1,len(data2)):
            dmax.append(sqrt(sum((data2[m] - data2[n]) ** 2 )))
    dmax=max(dmax)
    #a=np.percentile(dmax,10) #密度阈值(%10分位数)
    a=dmax*0.1
    k=7
    x1 = np.array([[1,1],[1,-1]])
    x2 = np.array([1,np.floor(k / 3)/ k])
    (Ps_,Pd_)=np.linalg.solve(x1,x2)
    H_ = -Ps_* np.log2(Ps_)-Pd_*np.log2(Pd_) #标签混合程度阈值

    data=np.array(df)
    ACC=[]
    for rate in [0.05,0.1,0.2,0.3,0.4]:
        acc_temp=[]
        for train_index, test_index in kf.split(data):
            train=data[train_index]
            test=data[test_index]
            train=pd.DataFrame(train)
            train.columns=df.columns
            Dn= []
            noise_set=pd.DataFrame(columns=train.columns)
            label_set=list(set(train.label))
            for r in label_set:    
                noise_set=pd.concat([noise_set,train[train.label==r].sample(frac=rate, replace=False)]) #选取噪声比例 
            train_cut=train[~train.index.isin(noise_set.index)]
            no=[]
            for j in range(0,len(noise_set)):
                no.append(np.random.choice(label_set))
            noise_set['label2']=no
            noise_set.reset_index(drop=True,inplace=True)
            for l in range(0,len(noise_set)):  #随机替换标签
                label_set=list(set(train.label))
                if (noise_set.loc[l,'label']==noise_set.loc[l,'label2']):
                    label_set.remove(noise_set.loc[l,'label'])
                    noise_set.loc[l,'label2']=np.random.choice(label_set) #从剩余列表中随机选择
            noise=noise_set.drop(['label'],axis=1)
            noise.rename(columns={'label2':'label'}, inplace = True)
            train= pd.concat([train_cut,noise],axis=0,ignore_index=True) #含噪声数据集
            train_set=np.array(train)
            kdt = KdTree()
            kdt.create(train_set)  
            for x in train_set:
                near,belong = kdt.search(x[:-1], k+1)  # 设置临近点的个数
                density = 0
                t,hon,hen,dist_hon,dist_hen,DoD,DRL= 0,0,0,0,0,0,0
                for n in near:
                    if x[-1] == n[1].data[-1]:
                        if (x != n[1].data).any():
                            t += 1
                            hon += 1
                            dist_hon += n[0]
                    else:
                        hen += 1
                        dist_hen += n[0]
                    density += n[0]  #密度   

                DoD = abs(dist_hen - dist_hon) #相异性差值(要加绝对值)
                DRL = (hen - hon) / k #标签异同差比    
                Ps = t / k
                Pd = 1 - Ps
                if (Ps==1)|(Ps==0):
                    H = 0
                else:
                    H = -Ps* np.log2(Ps)-Pd*np.log2(Pd)
              #  print('densit:',density)
                if (density <= a): #高密度区域
                    #print('高密度区')
                    if DRL > 0:
                        Dn.append(list(x))

                else: #低密度区域

                    if H >= H_: #低密度混合标签区域
                        DoD_ = density / k
                       # print('DoD阈值:',DoD_)
                        if DoD >= DoD_:
                            hon_sa,hen_sa,DRL_sa=0,0,0
                            near2=[]
                            for n in near:
                                if n[0] <= a: #该点到待测点距离小于a
                                    near2.append(n)
                            for n2 in near2:
                                if (x[-1]==n2[1].data[-1]):
                                    if (x != n[1].data).any():
                                        hon_sa +=1
                                else:
                                    hen_sa +=1
                            DRL_sa = hen_sa - hon_sa
                            if DRL_sa > 0:        
                                Dn.append(list(x))
                            elif DRL_sa ==0 and len(near2)<k:
                                if x[-1] != near[len(near2)][1].data[-1]:
                                    Dn.append(list(x)) 
                    else:#低密度单一标签区域
                        if DRL > 0:
                            Dn.append(list(x))
                            
            train_clean=train_set.tolist()
            for i in Dn:
                train_clean.remove(i)
            train_clean=pd.DataFrame(train_clean)
            train_clean.columns=df.columns
            test=pd.DataFrame(test)
            test.columns=df.columns
            rf_clf=RandomForestClassifier(n_estimators=10,random_state=42)    
            rf_clf.fit(train_clean.drop('label',axis=1),train_clean.label)
            y_pre=rf_clf.predict(test.drop('label',axis=1))
            acc_temp.append(accuracy_score(test.label, y_pre))   
        ACC.append(np.mean(acc_temp))
    return ACC

In [3]:
def RDS(df):
    data2=np.array(df.drop('label',axis=1))
    dmax=[]
    for m in range(0,len(data2)-1):
        for n in range(m+1,len(data2)):
            dmax.append(sqrt(sum((data2[m] - data2[n]) ** 2 )))
    a=np.percentile(dmax,10) #密度阈值(%10分位数)
    k=7
    x1 = np.array([[1,1],[1,-1]])
    x2 = np.array([1,np.floor(k / 3)/ k])
    (Ps_,Pd_)=np.linalg.solve(x1,x2)
    H_ = -Ps_* np.log2(Ps_)-Pd_*np.log2(Pd_) #标签混合程度阈值
    data=np.array(df)
    ACC=[]
    for rate in [0.05,0.1,0.2,0.3,0.4]:
        acc_temp=[]
        for train_index, test_index in kf.split(data):
            train=data[train_index]
            test=data[test_index]
            train=pd.DataFrame(train)
            train.columns=df.columns
            Dn,Dn_filted= [],[]
            noise_set=pd.DataFrame(columns=train.columns)
            label_set=list(set(train.label))
            for r in label_set:    
                noise_set=pd.concat([noise_set,train[train.label==r].sample(frac=rate, replace=False)]) #选取噪声比例 
            train_cut=train[~train.index.isin(noise_set.index)]
            no=[]
            for j in range(0,len(noise_set)):
                no.append(np.random.choice(label_set))
            noise_set['label2']=no
            noise_set.reset_index(drop=True,inplace=True)
            for l in range(0,len(noise_set)):  #随机替换标签
                label_set=list(set(train.label))
                if (noise_set.loc[l,'label']==noise_set.loc[l,'label2']):
                    label_set.remove(noise_set.loc[l,'label'])
                    noise_set.loc[l,'label2']=np.random.choice(label_set) #从剩余列表中随机选择
            noise=noise_set.drop(['label'],axis=1)
            noise.rename(columns={'label2':'label'}, inplace = True)
            train= pd.concat([train_cut,noise],axis=0,ignore_index=True) #含噪声数据集
            train_set=np.array(train) 
            kdt = KdTree()
            kdt.create(train_set)
            for x in train_set:
                near,belong = kdt.search(x[:-1], k+1)  # 设置临近点的个数
                density = 0
                t,hon,hen,dist_hon,dist_hen,DoD,DRL= 0,0,0,0,0,0,0
                for n in near:
                    if x[-1] == n[1].data[-1]:
                        if (x != n[1].data).any():
                            t += 1
                            hon += 1
                            dist_hon += n[0]
                    else:
                        hen += 1
                        dist_hen += n[0]
                    density += n[0]  #密度   

                DoD = abs(dist_hen - dist_hon) #相异性差值(要加绝对值)
                DRL = (hen - hon) / k #标签异同差比    
                Ps = t / k
                Pd = 1 - Ps
                if (Ps==1)|(Ps==0):
                    H = 0
                else:
                    H = -Ps* np.log2(Ps)-Pd*np.log2(Pd)
              #  print('densit:',density)
                if (density <= a): #高密度区域
                    #print('高密度区')
                    if DRL > 0:
                        Dn.append(list(x))

                else: #低密度区域

                    if H >= H_: #低密度混合标签区域
                        DoD_ = density / k
                       # print('DoD阈值:',DoD_)
                        if DoD >= DoD_:
                            train_hon,train_hen=[],[]
                            for j in train_set:
                                if j[-1]==x[-1]:
                                    train_hon.append(j)
                                else:
                                    train_hen.append(j)
                            kdt_hon=KdTree()
                            kdt_hen=KdTree()
                            kdt_hon.create(train_hon)
                            kdt_hen.create(train_hen)
                            near_hon,belong_hon=kdt_hon.search(x,k+1)
                            near_hen,belong_hen=kdt_hen.search(x,k)
                            density_hon,density_hen=0,0
                            for hon in near_hon:
                                density_hon += hon[0]
                            for hen in near_hen:
                                density_hen += hen[0]
                            if density_hon / density_hen > 1:
                                Dn.append(list(x))                 
                    else:#低密度单一标签区域
                        if DRL > 0:
                            Dn.append(list(x))

            noise_neighbor=[]                   
            for e in Dn:  # 计算噪声分数
                e=np.array(e)
                near3,belong3=kdt.search(e[:-1],k+1) #k近邻
                for n3 in near3:
                    noise_neighbor.append(list(n3[1].data))
            for e in Dn:
                e=np.array(e)
                near4,belong4=kdt.search(e[:-1],k+1)
                t_e=noise_neighbor.count(list(e))-1 #处于其它噪声近邻的次数
                confidence_e= 1 / np.sqrt(1+t_e*t_e)#e的置信度
                for i in range(1,k+1):
                    t_ei=noise_neighbor.count(list(near4[i][1].data))
                    confidence_ei=1 / np.sqrt(1+t_ei*t_ei) 
                    if near4[i][1].data[-1]==e[-1]:
                        different_class=-1
                    else:
                        different_class=1
                    if list(near4[i][1].data) in Dn: 
                        isnoise=1
                    else:
                        isnoise=-1
                    near5,belong5=kdt.search(near4[i][1].data[:-1],k+1)
                    neighborhood_e, dis_neighbor,dis_noise=0,0,0
                    for j in range(1,k+1):
                        dis_neighbor += near5[j][0]
                        if list(near5[j][1].data) in Dn:
                            dis_noise +=near5[j][0]
                    clean_ei= (dis_neighbor + isnoise*(dis_noise-dis_neighbor)) / (2*dis_neighbor) #纯净度
                    neighborhood_e += (clean_ei*confidence_ei*different_class)/k 
                NS_e=confidence_e * neighborhood_e #噪声得分
                if NS_e > 0:
                    Dn_filted.append(list(e))

                                     
            train_clean=train_set.tolist()
            for i in Dn_filted:
                train_clean.remove(i)
            train_clean=pd.DataFrame(train_clean)
            train_clean.columns=df.columns
            test=pd.DataFrame(test)
            test.columns=df.columns
            rf_clf=RandomForestClassifier(n_estimators=10,random_state=42)    
            rf_clf.fit(train_clean.drop('label',axis=1),train_clean.label)
            y_pre=rf_clf.predict(test.drop('label',axis=1))
            acc_temp.append(accuracy_score(test.label,y_pre))   
        ACC.append(np.mean(acc_temp))
    return ACC

In [4]:
import mpl_toolkits.axisartist.axislines as axislines
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib

Using matplotlib backend: Qt5Agg


In [5]:
def plot_res(res_DDF,res_RDS,title):
    noise_rate=[5,10,20,30,40]
    fig = plt.figure(figsize=(4,4))
    ax1 = axislines.Subplot(fig, 1,1,1)
    fig.add_subplot(ax1)
    ax1.set_xticks(noise_rate)
    ax1.axis([0,40,0,1])
    plt.plot(noise_rate,res_DDF,'r.-',label='DDF')
    plt.plot(noise_rate,res_RDS,'b*-.',label='DDF_SCORE')
    plt.xlabel('噪声比例/%')
    plt.ylabel('accuracy')
    plt.title(title)
    plt.legend()
    plt.show()

In [6]:
%%time
from sklearn.datasets import load_iris
iris = load_iris()
iris_sample= pd.DataFrame(iris.data, columns=iris.feature_names)
iris_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(iris_sample)))
iris_sample['label'] = iris.target
iris_DDF=DDF(iris_sample)
iris_RDS=RDS(iris_sample)
#plot_res(iris_DDF,iris_RDS,'(a)Iris 不同噪声下的准确率')

Wall time: 1min 12s


In [7]:
%%time
from sklearn.datasets import load_wine
wine= load_wine()
wine_sample = pd.DataFrame(wine.data, columns=wine.feature_names)
wine_sample = pd.DataFrame(min_max_scaler.fit_transform(np.array(wine_sample)))
wine_sample['label'] = wine.target
wine_DDF=DDF(wine_sample)
wine_RDS=RDS(wine_sample)
#plot_res(wine_DDF,wine_RDS,'(b)Wine 不同噪声下的准确率')

Wall time: 4min 24s


In [8]:
%%time
seeds=pd.read_csv('C:/Users/liu/python works/mydata/seeds.csv',engine='python')
seeds_sample=seeds.drop(['label'],axis=1)
seeds_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(seeds_sample)))
seeds_sample['label']=seeds.label
seeds_DDF=DDF(seeds_sample)
seeds_RDS=RDS(seeds_sample)
#plot_res(seeds_DDF,seeds_RDS,'(c)Seeds 不同噪声下的准确率')

Wall time: 1min 59s


In [9]:
%%time
glass=pd.read_csv('C:/Users/liu/python works/mydata/glass.csv')
glass_sample=glass.drop(['label'],axis=1)
glass_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(glass_sample)))
glass_sample['label']=glass.label
glass_DDF=DDF(glass_sample)
glass_RDS=RDS(glass_sample)
#plot_res(glass_DDF,glass_RDS,'(d)Glass 不同噪声下的准确率')

Wall time: 2min 40s


In [10]:
%%time
ecoli=pd.read_csv('C:/Users/liu/python works/mydata/ecoli.csv')
ecoli_DDF=DDF(ecoli)
ecoli_RDS=RDS(ecoli)
#plot_res(ecoli_DDF,ecoli_RDS,'(e)Ecoli 不同噪声下的准确率')

Wall time: 3min 54s


In [18]:
%%time
image=pd.read_csv('C:/Users/liu/python works/mydata/image.csv')
image_sample=image.drop(['label'],axis=1)
image_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(image_sample)))
image_sample['label']=image.label
image_DDF=DDF(image_sample)
image_RDS=RDS(image_sample)
#plot_res(image_DDF,image_RDS,'(f)Image 不同噪声下的准确率')

Wall time: 5min 44s


In [12]:
%%time
newthyroid=pd.read_csv('C:/Users/liu/python works/mydata/new-thyroid.csv',engine='python')
newthyroid_sample=newthyroid.drop(['label'],axis=1)
newthyroid_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(newthyroid_sample)))
newthyroid_sample['label']=newthyroid.label
newthyroid_DDF=DDF(newthyroid_sample)
newthyroid_RDS=RDS(newthyroid_sample)
#plot_res(newthyroid_DDF,newthyroid_RDS,'(g)Newthyroid 不同噪声下的准确率')

Wall time: 1min 58s


In [33]:
%%time
cmc=pd.read_csv('C:/Users/liu/python works/mydata/Contraception.csv',engine='python')
cmc_sample=cmc.drop(['label'],axis=1)
cmc_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(cmc_sample)))
cmc_sample['label']=cmc.label
cmc_DDF=DDF(cmc_sample)
cmc_RDS=RDS(cmc_sample)

Wall time: 2h 18min 18s


In [34]:
%%time
statlog=pd.read_csv('C:/Users/liu/python works/mydata/statlog.csv',engine='python')
statlog.drop_duplicates(inplace=True)
statlog_sample=statlog[['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18']]
statlog_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(statlog_sample)))
statlog_sample['label']=statlog.label  
statlog_DDF=DDF(statlog_sample)
statlog_RDS=RDS(statlog_sample)

Wall time: 1h 30min 36s


In [35]:
%%time
yeast=pd.read_csv('C:/Users/liu/python works/mydata/yeast.csv',engine='python')
yeast_sample=yeast.drop(['label'],axis=1)
yeast_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(yeast_sample)))
yeast_sample['label']=yeast.label
yeast_DDF=DDF(yeast_sample)
yeast_RDS=RDS(yeast_sample)

Wall time: 1h 19min 47s


In [36]:
%%time
vowel=pd.read_csv('C:/Users/liu/python works/mydata/vowel.csv',engine='python')
vowel_sample=vowel.drop(['label'],axis=1)
vowel_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(vowel_sample)))
vowel_sample['label']=vowel.label
vowel_DDF=DDF(vowel_sample)
vowel_RDS=RDS(vowel_sample)

Wall time: 1h 1min 35s


In [37]:
%%time
letter=pd.read_csv('C:/Users/liu/python works/mydata/letter.csv',engine='python')
letter_sample=letter.drop(['label'],axis=1)
letter_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(letter_sample)))
letter_sample['label']=letter.label
letter_DDF=DDF(letter_sample)
letter_RDS=RDS(letter_sample)

Wall time: 4h 8min 27s


In [38]:
ACC_DDF=np.array(iris_DDF)+np.array(wine_DDF)+np.array(seeds_DDF)+np.array(glass_DDF)+np.array(ecoli_DDF)+np.array(image_DDF)+np.array(newthyroid_DDF)+np.array(cmc_DDF)+np.array(statlog_DDF)+np.array(yeast_DDF)+np.array(vowel_DDF)+np.array(letter_DDF)
ACC_RDS=np.array(iris_RDS)+np.array(wine_RDS)+np.array(seeds_RDS)+np.array(glass_RDS)+np.array(ecoli_RDS)+np.array(image_RDS)+np.array(newthyroid_RDS)+np.array(cmc_RDS)+np.array(statlog_RDS)+np.array(yeast_RDS)+np.array(vowel_RDS)+np.array(letter_RDS)

In [39]:
plot_res(ACC_DDF/12,ACC_RDS/12,'不同噪声下的准确率')